In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from finance_byu.regtables import Regtable
import warnings
import matplotlib.pyplot as plt

In [ ]:
# Load data into pandas dataframe
df = pd.read_parquet('Data/BoQ_Data.parquet')
df.head()

In [54]:
def AddMarketReturn(dataframe):
    """Adds a market return variable (mkt) to the dataframe"""

    # Define a function to calculate the weighted mean return
    def weighted_mean(group):
        return (group['RET'] * group['ME']).sum() / group['ME'].sum()

    # Group by 'year' and apply the weighted mean to each group
    dataframe['mkt'] = dataframe.groupby('DATE').apply(
        lambda group: pd.Series(weighted_mean(group), index=group.index)
    ).reset_index(drop=True)

    return dataframe

In [ ]:
df = AddMarketReturn(df)